In [1]:
import copy
import pandas as pd
import numpy as np
import json


import datetime as dt
import random
import time

import math
import bisect
import itertools

#import statsmodels.api as sm

from IPython.display import clear_output


import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from graphviz import Digraph
from scipy.ndimage import shift
#import pydot


from getpass import getpass
import os

import TransEntropy_mod as te_mod
import k2_alg as k2

Requirement already up-to-date: pip in /opt/conda/lib/python3.7/site-packages (19.3.1)


In [2]:
def graph_simple(df, eng = 'dot', lbl=False, size='3,3'):
    edge_style = ""
    g = Digraph(engine=eng)
    
    g = Digraph(engine=eng)
    g.attr(ratio='fill', margin='0')
    
    in_graph = []
    for k, row in enumerate(df.index):
        if any(df.loc[row]):
            g.node(str(row),row, shape='oval', fontsize='10', width='0', style='filled', fillcolor='white', color="black")
            in_graph.append(row)

              
    for c, col in enumerate(df.columns):
        if any(df[col]):
            if col not in in_graph:
                g.node(str(col), col, shape='oval', fontsize='10', width='0', style='filled', fillcolor='white', color="black") 

    for j, col in enumerate(df.columns):
        for i, row in enumerate(df.index):
            if(df[col][i]):
                if lbl:
                    label=str(df.at[row,col])
                else:
                    label = ''
                g.edge(str(row), str(col), label=label, style= edge_style, color='black')  
    return g 

In [3]:
def get_lags_ances_df(mat,idx, soma, dict_lags, lista, dict_caminhos):
    lista.append(idx)
    if np.all(mat[idx] == np.zeros(len(mat))):
        return [dict_lags,dict_caminhos]
    for i,dad_lag in enumerate(mat[idx]):
        if dad_lag > 0:
          
            soma += dad_lag
            try:
                dict_lags[mat.columns[i]].append(soma)
                dict_caminhos[mat.columns[i]].append(lista)
            except:
                dict_lags[mat.columns[i]] = [soma]
                dict_caminhos[mat.columns[i]]= [lista]
        
            get_lags_ances_df(mat, mat.columns[i], soma, dict_lags, lista[:], dict_caminhos)
            soma -= dad_lag
            
    return [dict_lags, dict_caminhos]

In [4]:
def get_all_shifts(nodes, mat):
    dic = {}
    for node in nodes:
        mat_cp = mat.copy()
#         if not np.all(mat_cp[node] == np.zeros(len(mat_cp))):
        dic[node] = get_lags_ances_df(mat_cp, node, 0, {}, [],{})[0]
    return dic

In [5]:
#reading of alarms, te, and lags

df_alm_mean5 = pd.read_csv("resultados_defesa/alm_seq_m5.csv", index_col=0)

df_te = pd.read_csv("resultados_defesa/df_te_i3eA.csv", index_col=0)
df_te_lags = pd.read_csv("resultados_defesa/df_te_lag_i3eA.csv", index_col=0)

df_te_i3eA_ac = pd.read_csv('resultados_defesa/te_i3eA_ac.csv', index_col=0)
df_lags_final = pd.read_csv('resultados_defesa/df_lags_i3eA_final.csv', index_col=0)

In [6]:
#genereation of k2_tree
dict_all_lags = get_all_shifts(df_lags_final.columns, df_lags_final.copy())
k2_te_tree = te_mod.gen_tree_from_lags(dict_all_lags)

In [7]:
#k2_result_05 = k2.k2(df_alm_mean5, k2_tree, df_lags_final, 0.7)

In [8]:
c = np.arange(0.1,1.01,.1)
c

array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

In [9]:
#search of k2 parameter

c = np.arange(0.1,1.01,.1)

for param in c[1:]:
    result = k2.k2(df_alm_mean5, k2_te_tree, df_lags_final, round(param,1))
    clean_df = k2.gen_clean_df(df_lags_final, result)
    df_path  = df_lags_final[clean_df>0].fillna(0)

    g_k2 = graph_simple(df_path, eng='dot', size='',lbl=False)
    g_k2.render('resultados_defesa/k2_' + str(round(param,1)) , view=False) 
    
    with open ('k2_'+ str(round(param,1)) +'.txt', 'w') as file:
        file.write(json.dumps(result))
    
    
        

here  x21_high x01_high
here  x21_high x01_high
here  x07_low x01_high
here  x21_high x01_high
here  x09_high x07_high
here  x06_low x07_high
here  x21_high x07_high
here  x21_high x07_high
here  x21_high x07_high
here  x21_high x07_high
here  x21_high x07_high
here  x21_high x07_high
here  x21_high x07_high
here  x21_high x07_high
here  x21_high x07_high
here  x21_high x07_high
here  x21_high x07_high
here  x21_high x07_high
here  x21_high x08_high
here  x07_low x08_high
here  x21_high x08_high
here  x21_high x08_high
here  x21_high x09_high
here  x07_low x09_high
here  x01_low x09_high
here  x01_low x09_high
here  x21_high x06_low
here  x21_high x07_low
here  x09_high x21_low
here  x21_high x01_high
here  x21_high x01_high
here  x07_low x01_high
here  x21_high x01_high
here  x09_high x07_high
here  x06_low x07_high
here  x21_high x07_high
here  x21_high x07_high
here  x21_high x07_high
here  x21_high x07_high
here  x21_high x07_high
here  x21_high x07_high
here  x21_high x07_high
her